In [17]:
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import math

## Algoritmos ##

### Branch and bound ###

In [18]:
class Node:
    def __init__(self, bound, boundEdges, cost, solution):
        self.bound = bound
        self.boundEdges = boundEdges
        self.cost = cost
        self.solution = solution
    
    def __lt__(self, other):
        if len(self.solution) == len(other.solution):
            return self.bound < other.bound
        return len(self.solution) > len(other.solution)
    def __repr__(self) -> str:
        return f"Node({self.bound}, {self.boundEdges}, {self.cost}, {self.solution})"

def findTwoMinimalEdges(list):
    min1 = np.inf
    min2 = np.inf
    for j in list:
        if list[j]['weight'] < min1:
            min2 = min1
            min1 = list[j]['weight']
        elif list[j]['weight'] < min2:
            min2 = list[j]['weight']
    return min1, min2

def findInitialBound(A):
    bound = 0
    initialBoundEdges = np.zeros((A.number_of_nodes(), 2), dtype=list)
    for i in range(A.number_of_nodes()):
        min1, min2 = findTwoMinimalEdges(A[i])
        initialBoundEdges[i][0] = min1
        initialBoundEdges[i][1] = min2
        bound += min1 + min2
    return bound / 2, initialBoundEdges

def findBound(A, solution, boundEdges, bound):
    changedEdges = np.zeros(A.number_of_nodes(), dtype=int)
    newEdges = np.array(boundEdges)
    edgeWeight = A[solution[-2]][solution[-1]]['weight']
    sum = bound * 2
    if newEdges[solution[-2]][0] != edgeWeight:
        if changedEdges[solution[-2]] == 0:
            sum -= newEdges[solution[-2]][1]
            sum += edgeWeight
        else:
            sum -= newEdges[solution[-2]][0]
            sum += edgeWeight
        changedEdges[solution[-2]] += 1
    if newEdges[solution[-1]][0] != edgeWeight:
        if changedEdges[solution[-1]] == 0:
            sum -= newEdges[solution[-1]][1]
            sum += edgeWeight
        else:
            sum -= newEdges[solution[-1]][0]
            sum += edgeWeight
        changedEdges[solution[-1]] += 1
    return sum / 2, newEdges
from heapq import heappush, heappop

def branchAndBoundTSP(A):
    initialBound, initialBoundEdges = findInitialBound(A)
    root = Node(initialBound, initialBoundEdges, 0, [0])
    heap = []
    heappush(heap, root)
    best = np.inf
    solution = []
    nodeCount = 0
    while heap:
        node = heappop(heap)
        nodeCount += 1
        level = len(node.solution)
        if level > A.number_of_nodes():
            if best > node.cost:
                best = node.cost
                solution = node.solution
        else:
            if node.bound < best:
                if level < A.number_of_nodes() - 2:
                    for k in range(1, A.number_of_nodes()):
                        if k == node.solution[-1] or k == 0:
                            continue
                        edgeWeight = A[node.solution[-1]][k]['weight']
                        newBound, newEdges = findBound(A, node.solution + [k], node.boundEdges, node.bound) 
                        if k not in node.solution and newBound < best:
                            newNode = Node(newBound, newEdges, node.cost + edgeWeight, node.solution + [k])
                            if k == 2:
                                if 1 not in node.solution:  
                                    continue 
                            heappush(heap, newNode)
                else:
                    for k in range(1, A.number_of_nodes()):
                        if k == node.solution[-1] or k == 0:
                            continue
                        lastNode = 0
                        for i in range(1, A.number_of_nodes()):
                            if i not in node.solution + [k] and k != i:
                                lastNode = i
                                break
                        edgeWeight = A[node.solution[-1]][k]['weight']
                        nextEdgeWeight = A[k][lastNode]['weight']
                        lastEdgeWeight = A[lastNode][0]['weight']
                        cost = node.cost + edgeWeight + nextEdgeWeight + lastEdgeWeight
                        if k not in node.solution and cost < best:
                            newNode = Node(cost, [], cost, node.solution + [k, lastNode, 0])
                            heappush(heap, newNode)
    return best, solution

### Twice around the tree ###

In [19]:
def twiceAroundTheTreeTSP(A):
    MST = nx.minimum_spanning_tree(A)
    path = list(nx.dfs_preorder_nodes(MST, 1))
    path.append(path[0])
    weight = sum(A[path[i]][path[i + 1]]['weight'] for i in range(len(path) - 1))

    return weight, path

### Christofides ###

In [20]:
def shortcutPath(A):
    path = [x[0] for x in nx.eulerian_circuit(A, 1)]
    return list(dict.fromkeys(path)) + [path[0]]

def christofidesTSP(A):
    MST = nx.minimum_spanning_tree(A)
    oddnodes = [node for node, degree in nx.degree(MST) if degree % 2 == 1]
    pairing = nx.min_weight_matching(nx.subgraph(A, oddnodes))

    mst = nx.MultiGraph(MST)
    mst.add_edges_from((node1, node2, A[node1][node2]) for node1, node2 in pairing)

    path = shortcutPath(mst)
    peso = sum(A[path[i]][path[i + 1]]['weight'] for i in range(len(path) - 1))

    return peso, path

## Testes ##

### Funções para cálculo de métricas

In [21]:
from memory_profiler import memory_usage
import time
import numpy as np
import csv

def runAlgorithm(algorithm, data, best_weight):
    start_time = time.time()
    end_time = 30 * 60  # 30 minutos em segundos
    result_dict = {
        'algorithm': algorithm.__name__,
        'weight': None,
        'runtime': None,
        'memory_use': None,
        'deviation': None
    }

    try:
        memory_use = memory_usage((algorithm, (data,)))
        result = algorithm(data)
        tempo_fim = time.time()

        runtime = tempo_fim - start_time
        if runtime > end_time:
            raise TimeoutError

        weight_algorithm = result[0]
        deviation = ((weight_algorithm - best_weight) / best_weight) * 100

        result_dict.update({
            'weight': weight_algorithm,
            'runtime': runtime,
            'memory_use': np.max(memory_use),
            'deviation': deviation
        })

        return result_dict
    except TimeoutError:
        result_dict['weight'] = "NA"
        return result_dict
    
def save_csv(data, filename, dataset_name, city_count):
    with open(filename, mode='a', newline='') as file:
        writer = csv.writer(file)
        # Escreve o cabeçalho se o arquivo estiver vazio
        if file.tell() == 0:
            writer.writerow(['dataset', 'number of cities', 'algorithm', 'weight', 'runtime', 'memory use', 'deviation'])

        writer.writerow([dataset_name, city_count, data['algorithm'], data['weight'],
                         data['runtime'], data['memory_use'], data['deviation']])

### Funções para carregamento dos datasets e montagem dos grafos

In [22]:
import tsplib95

def load_tsp(caminho_arquivo):
    problema = tsplib95.load(caminho_arquivo)
    grafo = problema.get_graph()
    return grafo

def load_best(caminho_arquivo):
    with open(caminho_arquivo, 'r') as file:
        linhas = file.readlines()
        tour = [int(linha.strip()) for linha in linhas if linha.strip().isdigit()]
    return tour

def find_best_weight(grafo, tour_optima):
    peso_total = 0
    for i in range(len(tour_optima) - 1):
        peso_total += grafo[tour_optima[i]][tour_optima[i + 1]]['weight']
    peso_total += grafo[tour_optima[-1]][tour_optima[0]]['weight']
    return peso_total

### Executa testes nos datasets

In [23]:

import os 

datasets = os.listdir('../Algoritmos2-TrabalhoPr-tico2/datasets')
datasets = [f for f in datasets if f.endswith('.tsp')]
datasets = [f.replace('.tsp', '') for f in datasets]

csv_name = 'results_tsp.csv'

for i in range(len(datasets)):

    dataset_name = datasets[i]

    graph = load_tsp(f'../Algoritmos2-TrabalhoPr-tico2/datasets/{dataset_name}.tsp')
    best_tour = load_best(f'../Algoritmos2-TrabalhoPr-tico2/datasets/{dataset_name}.opt.tour')
    best_weight = find_best_weight(graph, best_tour)

    num_cities = len(graph.nodes)

    if num_cities <= 15:
        print(f'Running {dataset_name} with {num_cities} cities, branch and bound')
        result = runAlgorithm(branchAndBoundTSP, graph, best_weight)
        save_csv(result, csv_name, dataset_name, num_cities)

    for algorithm in [twiceAroundTheTreeTSP, christofidesTSP]:
        print(f'Running {dataset_name} with {num_cities} cities, {algorithm.__name__}')
        result = runAlgorithm(algorithm, graph, best_weight)
        save_csv(result, csv_name, dataset_name, num_cities)

Running berlin52 with 52 cities, twiceAroundTheTreeTSP
Running berlin52 with 52 cities, christofidesTSP
Running kroA100 with 100 cities, twiceAroundTheTreeTSP
Running kroA100 with 100 cities, christofidesTSP
Running pa561 with 561 cities, twiceAroundTheTreeTSP
Running pa561 with 561 cities, christofidesTSP
Running pcb442 with 442 cities, twiceAroundTheTreeTSP
Running pcb442 with 442 cities, christofidesTSP
Running tsp225 with 225 cities, twiceAroundTheTreeTSP
Running tsp225 with 225 cities, christofidesTSP


### Análise de resultados e plotagem de gráficos


In [28]:
import pandas as pd
import matplotlib.pyplot as plt
import os

if not os.path.exists('img'):
    os.makedirs('img')

df = pd.read_csv('./results_tsp.csv')

def plot_results(df, y_col, y_label, title, file_name):
    plt.figure(figsize=(12, 8))
    for algorithm in df['algorithm'].unique():
        df_alg = df[df['algorithm'] == algorithm]
        df_grouped = df_alg.groupby('number of cities')[y_col].mean()
        plt.plot(df_grouped, label=algorithm, marker='o', linestyle='dashed', linewidth=2, markersize=8)
    plt.xlabel('dataset size (number of cities)')
    plt.ylabel(y_label)
    plt.title(title)
    plt.legend()
    plt.savefig(f'img/{file_name}.png')
    plt.close()

plot_results(df, 'runtime', 'mean runtime (s)', 
               'runtime by dataset size', 'runtime')

plot_results(df, 'memory use', 'mean memory use (MB)', 
               'memory use by dataset size', 'memory_use')

for algorithm in df['algorithm'].unique():
    mean_deviation = df[df['algorithm'] == algorithm]['deviation'].mean()
    print(f"mean deviation for {algorithm}: {mean_deviation:.2f}%")

mean deviation for twiceAroundTheTreeTSP: 34.70%
mean deviation for christofidesTSP: 16.47%
